# Profitable App Categories For The App Store & Google Play Markets

**Objective: Analyze mobile apps available on App Store and Google Play Markets. Decide which category of apps is most profitable in order to help developers build a free app that attracts users and drives revenue based on in-app ads.** 



## Retreiving & Converting The Data 

For this project we will be working with two datasets from [kaggle](https://www.kaggle.com).

[Google Play Store Apps](https://www.kaggle.com/lava18/google-play-store-apps/home) - contains data about approximately ten thousand Android apps from Google Play
<br>
[Mobile App Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) - contains data about approximately seven thousand iOS apps from the App Store

In order to refer to our datasets with ease in our analysis, we split the header from the rest of the data for both datasets.

In [1]:
from csv import reader

###Google Data###

opened_file = open('/Users/SohailMansouri/Downloads/google-play-store-apps/googleplaystore.csv')
read_file = reader(opened_file)
google = list(read_file)
google_header = google[0]
google_data = google[1:]

###Apple Data###
opened_file = open('/Users/SohailMansouri/Downloads/app_store_data/AppleStore.csv')
read_file = reader(opened_file)
apple = list(read_file)
apple_header = apple[0]
apple_data = apple[1:]

Knowing that we'll need to analyze certain rows in our datasets frequently, we write a function, *explore_data()*, that allows us to display specified rows of a dataset with ease. We also add the option to see the number of rows & columns when the *rows_and_columns* argument is set to *True*. 


In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') 

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

## Removing Incorrect Data

As we begin the process of cleaning our dataset, we notice from the [kaggle discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) where we retreived our dataset that in our Google dataset is an entry (10472) that is missing a value, causing the rest of the data for that entry to become shifted. We explore this entry below and delete it from our dataset. 

In [3]:
explore_data(google_data,10472,10473)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']




In [4]:
del google_data[10472] #delete the Life Made App that had incorrect data. Only run this once.

## Removing Duplicate Entries

After exploring the Google Play dataset, we find that much of our data consists of duplicate entries. We want to get an idea for which entries are duplicates and how many of them there are, so we write a for loop that analyzes our Google data and extracts the duplicates in the list *duplicate_apps*, while all unique apps are placed into the list *unique_apps*

In [5]:
duplicate_apps = []
unique_apps = []

for i in google_data:
    name = i[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    unique_apps.append(name)

print(len(duplicate_apps))

1181


From the output above we can see that there are 1181 apps that are duplicates.

<br>
We need to remove the duplicate entries, however, we notice that not all of these duplicated apps are identical. After examining the dataset, we notice that the fourth position of each entry corresponds to the number of reviews for that app, and that this number is different for each duplicate. We want to keep only the entry that has the highest number of reviews, as that is most likely the most recent entry!

In [6]:
reviews_max = {}

for i in google_data:
    name = i[0]
    n_reviews = float(i[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print(len(reviews_max))

        
        

9659


The above code is the first step in tackling the problem of duplicates. We iterate over the google data and assign to the reviews_max dictionary the unique apps as keys and only the highest number of ratings available for that app as the key value. 

We then continue our journey by using the reviews_max dictionary to remove duplicates. 

1. We start by initializing two empty lists, android_clean and already_added.
2. We loop through the android data set, and for every iteration:
   * We isolate the name of the app and the number of reviews.
   * We add the current row (i) to the android_clean list, and the app name (name) to the already_cleaned list if:
     * The number of reviews of the current app matches the number of reviews of that app as described in the      reviews_max dictionary; 
     * and The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

In [7]:
android_clean = []
already_added = []

for i in google_data:
    name = i[0]
    n_reviews = float(i[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(i)
        already_added.append(name)
        
print(len(android_clean))

9659


## Removing Non English Apps

Now that we have unique apps in both our ios and google apps, we continue our our data cleanse journey by targeting and removing any non-english apps. In the code below we see some a few samples of the apps that are not in english and need to be removed.

In [8]:
print(apple_data[2862][2])
print(apple_data[2974][2])

print(android_clean[4412][0])
print(android_clean[7940][0])

ニート勇者３ -闇の側の者たち- 無料ロールプレイングゲームRPG
Creema（クリーマ）- ハンドメイドマーケットプレイス
中国語 AQリスニング
لعبة تقدر تربح DZ


One method we can use to remove the non-english apps is to target only the apps that are comprised of the english alphabet, numbers 0-9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

Interestingly, each of characters in the english language are encoded using the ASCII standard and have a corresponding number between 0 and 127. This means that any non-english characters will be above this range.

We start by building out our *is_english* function which takes a string, and checks if the characters of the string are above 127. If they are, we return false, indicating that the string is not in english. We print out a few test runs with our function to determine its legitimacy. 

In [9]:
def is_english(string):
    for i in string:
        if ord(i) > 127:
            return False 
    return True
        

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
        

True
False
False
False


From the above output, we can see that our function has deemed some app names as not english (False) even though we clearly see that they are. We refine our function by adding in a condition that will allow up to 3 non-english characters before declaring the input as non_english.

In [10]:
def is_english(string):
    non_english_count = 0
    
    for i in string:
        if ord(i) > 127:
            non_english_count += 1
    
    if non_english_count > 3:
        
            return False
    else:
            return True


        

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
True
True


Great, the function seems to be working. we want to apply this to our datasets. Below we create two new lists and will use a for loop with our *is_english* function to append the correct apps. We do this for both our google data (which is now the list *android_clean* since we modified our original google data) and apple_data.

In [14]:
google_english_apps = []
ios_english_apps = []

for i in android_clean:
    name= i[0]
    if is_english(name) == True:
        google_english_apps.append(i)

        
for i in apple_data:
    name= i[2]
    if is_english(name) == True:
        ios_english_apps.append(i)
        
        
print("There are " + str(len(google_english_apps)) + " apps in our google dataset.")
print("There are " + str(len(ios_english_apps)) + " apps in our ios dataset.")  


    

There are now 9614 apps in our google dataset.
There are now 6183 apps in our apple dataset.


In [34]:
google_free_apps = []
ios_free_apps = []

for i in google_english_apps:
    app_type = i[7]
    if app_type == '0':
        google_free_apps.append(i)
        
for i in ios_english_apps:
    price = i[5]
    if price == '0':
        ios_free_apps.append(i)     
        
print("There are " + str(len(google_free_apps)) + " free apps in our google dataset. ")
print("There are " + str(len(ios_free_apps)) + " free apps in our apple dataset. ")


There are 8864 free apps in our google dataset. 
There are 3222 free apps in our apple dataset. 


## Most Common Apps by Genre
### Part One

As we mentioned before, our goal is to identify the most profitable app profiles since our new app's success will depend largely on the number of people that use it. 
<br>
<br>
Once we have gathered the necessary insight regarding what type of app to build, we implement the following steps:
<ol>
    <li>Build a minimal Android version of the app, and add it to Google Play.</li>
    <li>If the app has a good response from users, we then develop it further.</li>
    <li>If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.</li>
</ol>
<br>
Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. 
<br>
<br>


### Part Two
Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set. Below, we define a function *freq_table* that takes a dataset and index and returns a table of percentages based on the frequency of each key (in our case, the column corresponding to genre). We then define a second function, *display_table* that also takes a dataset and index as inputs. Our *display_table* function utilizes our *freq_table* function to create a list of tuples. It will then sort the list in a descending order. 

In [45]:
def freq_table(dataset,index):
    
    table = {}
    total = 0
    
    for i in dataset:
        total += 1
        key = i[index]
        if key in table:
            table[key] += 1
        else:
            table[key] = 1
            
    table_percentages = {}
    
    for key in table:
        percentage = (table[key]/total) * 100
        table_percentages[key] = percentage
        
    return table_percentages

def display_table(dataset,index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key],key)
        table_display.append(key_val_as_tuple)
    table_sorted = sorted(table_display, reverse = True)
    
    for i in table_sorted:
        print(i[1], ":",i[0])
        

### Part Three

Let's use our newly created function to identify the most common apps in the iOS app store.

In [46]:
display_table(ios_free_apps,12)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


By analyzing our frequency table above, we see that more than half of the apps in the app store are games! The next genre following games in popularity is entertainment, which is less than 8% of total apps. One important thing to note here is that just because there are alot more games, that doesn't mean that there are necessarily greater amount of users for those type of apps compared to other genres. We continue our analysis by exploring the "Category" and "Genre" columns in our google dataset. 

In [47]:
display_table(google_free_apps,1) #Category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [48]:
display_table(google_free_apps,-4) #Genres

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

From the two tables above, we see that in our google dataset, most apps fall under the *Family* (18%) category. Upon further investigation we see that most of this data is actually games for kids! While inspecting the genre table, we see that *Tools* is the most common genre, indicating that much of the google playstore apps are designed for practically. It's interesting to compare these results to those of the ios apps frequency table, which showed us that over half of the apps in the appstore are games. Additionally, it's important to note that the *entertainment* genre is popular among both ios appstore and google playstore datasets.

## Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot column.
<br>
<br>
Below, we calculate the average number of user ratings per app genre on the App Store:

In [60]:
genres_ios = freq_table(ios_free_apps,-5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_free_apps:
        genre_app = app[-5]
        if genre_app == genre:
            num_user_ratings = float(app[6])
            total += num_user_ratings
            len_genre += 1

    avg_num_user_ratings = (total / len_genre)
    print(genre,":", avg_num_user_ratings)
    

Productivity : 21028.410714285714
Weather : 52279.892857142855
Shopping : 26919.690476190477
Reference : 74942.11111111111
Finance : 31467.944444444445
Music : 57326.530303030304
Utilities : 18684.456790123455
Travel : 28243.8
Social Networking : 71548.34905660378
Sports : 23008.898550724636
Health & Fitness : 23298.015384615384
Games : 22788.6696905016
Food & Drink : 33333.92307692308
News : 21248.023255813954
Book : 39758.5
Photo & Video : 28441.54375
Entertainment : 14029.830708661417
Business : 7491.117647058823
Lifestyle : 16485.764705882353
Education : 7003.983050847458
Navigation : 86090.33333333333
Medical : 612.0
Catalogs : 4004.0


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [63]:
for app in ios_free_apps:
    if app[-5] == 'Navigation':
        print(app[2], ":", app[6])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Geocaching® : 12811
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5
CoPilot GPS – Car Navigation & Offline Maps : 3582
Google Maps - Navigation & Transit : 154911


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.
Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.
Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [64]:
for app in ios_free_apps:
    if app[-5] == 'Reference':
        print(app[2], ":", app[6])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
Merriam-Webster Dictionary : 16849
Google Translate : 26786
Night Sky : 12122
WWDC : 762
Jishokun-Japanese English Dictionary & Translator : 0
教えて!goo : 0
VPN Express : 14
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Real Bike Traffic Rider Virtual Reality Glasses : 8


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.
<br>
<br>
This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.
<br>
<br>
Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:
<br>
* Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.
* Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.
* Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.
<br>
<br>
Now let's analyze the Google Play market a bit.

In [65]:
display_table(google_free_apps,5) #Looking at the number of installs

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343



One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.
<br>
<br>
We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.
<br>
<br>
To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [67]:
google_categories = freq_table(google_free_apps,1)

for category in google_categories:
    total=0
    len_category=0
    for app in google_free_apps:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace('+',"")
            n_installs = n_installs.replace(',','')
            n_installs = float(n_installs)
            total += n_installs
            len_category += 1
    avg_num_installs = (total / len_category) * 100
    print(category,":",avg_num_installs)
    

ART_AND_DESIGN : 198633508.77192983
AUTO_AND_VEHICLES : 64731781.70731707
BEAUTY : 51315188.679245286
BOOKS_AND_REFERENCE : 876781189.4736841
BUSINESS : 171229014.74201474
COMICS : 81765727.27272727
COMMUNICATION : 3845611916.7247386
DATING : 85402883.03030303
EDUCATION : 183349514.5631068
ENTERTAINMENT : 1164070588.235294
EVENTS : 25354222.22222222
FINANCE : 138769247.5609756
FOOD_AND_DRINK : 192489773.63636363
HEALTH_AND_FITNESS : 418882198.5347985
HOUSE_AND_HOME : 133154056.16438356
LIBRARIES_AND_DEMO : 63850373.4939759
LIFESTYLE : 143781626.87861273
GAME : 1558801560.324826
FAMILY : 369564181.98090696
MEDICAL : 12055061.980830671
SOCIAL : 2325365212.7118645
SHOPPING : 703687731.1557789
PHOTOGRAPHY : 1784011040.2298849
SPORTS : 363864014.28571427
TRAVEL_AND_LOCAL : 1398407771.0144928
TOOLS : 1080139129.8666668
PERSONALIZATION : 520148261.22448975
PRODUCTIVITY : 1678733134.4927535
PARENTING : 54260362.06896552
WEATHER : 507448619.7183099
VIDEO_PLAYERS : 2472787245.2830186
NEWS_AND_MA

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [72]:
for app in google_free_apps:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+' 
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0],":",app[5])
                                    

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced by more than ten times:

In [75]:
under_100_m = []

for app in google_free_apps:
    n_installs = app[5]
    n_installs = n_installs.replace('+','')
    n_installs = n_installs.replace(',','')
    if float(n_installs) < 100000000:
        under_100_m.append(float(n_installs))
        
sum(under_100_m)/len(under_100_m)

2821347.6231027693

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).
<br>
<br>
Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.
The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.
<br>
<br>
The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.
<br>
<br>
Let's take a look at some of the apps from this genre and their number of installs:

In [84]:
for app in google_free_apps:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+' 
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0],":",app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


It looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [85]:
for app in google_free_apps:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+' 
                                      or app[5] == '5,000,000+'
                                      or app[5] == '10,000,000+'
                                      or app[5] == '50,000,000+'):
        print(app[0],":",app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.


## Conclusion

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.
<br>
<br>
We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.